In [15]:
!pip install streamlit requests cohere python-dotenv deep-translator

In [16]:

import pickle
import requests
from urllib.parse import quote
import cohere
import os
from dotenv import load_dotenv
from deep_translator import GoogleTranslator

In [17]:

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
TMDB_API_KEY = os.getenv('TMDB_API_KEY')  # Make sure to set this in your .env file

In [18]:

folder_path = r'E:\#CODING_Learining_PATH\Movie_Recommendation_System_Project\MRS\PJ_withoutLLMS\Data'  # Adjust the path as needed
# Load the files from the specified folder
try:
    with open(os.path.join(folder_path, 'movies_list_update.pkl'), 'rb') as file:
        movies = pickle.load(file)
    with open(os.path.join(folder_path, 'similarity.pkl'), 'rb') as file:
        similarity = pickle.load(file)

    movies_list = movies['title'].values
    print("Files loaded successfully!")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except pickle.PickleError as e:
    print(f"Error loading pickle file: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: No module named 'numpy._core'


In [19]:
movies_list = movies['title'].values

In [20]:

# Load movies and similarity data

# Configure Cohere API
co = cohere.Client(COHERE_API_KEY)

def fetch_movie_url(movie_title):
    cleaned_title = movie_title.replace(' ', '-').replace('&', '').replace(':', '')
    encoded_title = quote(cleaned_title.lower())
    return f"https://www.channelmyanmar.to/{encoded_title}/"

In [21]:

def fetch_poster(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}&language=en-US"
    response = requests.get(url).json()
    poster_path = response.get('poster_path', '')
    if poster_path:
        return f"https://image.tmdb.org/t/p/w500{poster_path}"
    return "https://via.placeholder.com/500x750?text=No+Poster+Available"

In [22]:

def recommend(selected_movie_title):
    index = movies[movies['title'] == selected_movie_title].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommend_movie = []
    recommend_poster = []
    movie_urls = []
    for i in distances[0:5]:
        movie_id = movies.iloc[i[0]].id
        movie_title = movies.iloc[i[0]].title
        recommend_movie.append(movie_title)
        recommend_poster.append(fetch_poster(movie_id))
        movie_urls.append(fetch_movie_url(movie_title))
    return recommend_movie, recommend_poster, movie_urls

In [23]:
def fetch_popular_movies():
    url = f"https://api.themoviedb.org/3/movie/popular?api_key={TMDB_API_KEY}&language=en-US"
    response = requests.get(url)
    if response.status_code != 200:
        st.error(f"Failed to fetch popular movies. Status code: {response.status_code}")
        return []
    data = response.json()
    popular_movies = data.get('results', [])
    return [(movie['title'], movie['id'], movie.get('poster_path', '')) for movie in popular_movies]

In [24]:
import streamlit as st
# Streamlit UI
st.title("AI Powered Movie Recommendation System")

# Inject custom CSS
st.markdown(
    """
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');
    .poster-container {
        display: inline-block;
        position: relative;
        cursor: pointer;
        margin: 10px;
    }

    .poster-container img {
        transition: transform 0.3s ease;
        border-radius: 10px;
    }

    .poster-container img:hover {
        transform: scale(1.1);
    }

    .poster-container .poster-title {
        position: absolute;
        font-family: 'Roboto', sans-serif;
        bottom: 0;
        left: 0;
        width: 100%;
        background: rgba(0, 0, 0, 0.6);
        color: white;
        text-align: center;
        padding: 5px;
        border-top: 0.5px solid #00f1ff;
        border-bottom: 0.5px solid #00f1ff;
        border-radius: 0 0 10px 10px;
        opacity: 0;
        transition: opacity 0.3s ease;
    }

    .poster-container:hover .poster-title {
        opacity: 1;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Sidebar for popular movies
st.sidebar.header('Popular Movies')
popular_movies = fetch_popular_movies()
if popular_movies:
    for title, movie_id, poster_path in popular_movies:
        poster_url = "https://image.tmdb.org/t/p/w500/" + poster_path if poster_path else "https://via.placeholder.com/500x750?text=No+Poster+Available"
        movie_url = fetch_movie_url(title)
        st.sidebar.markdown(
            f'<div class="poster-container">'
            f'<a href="{movie_url}" target="_blank">'
            f'<img src="{poster_url}" width="200" height="250" />'
            f'</a>'
            f'<div class="poster-title">{title}</div>'
            f'</div>',
            unsafe_allow_html=True
        )
else:
    st.sidebar.write("No popular movies available.")

# Select movie from the database
select_value = st.selectbox('Select Movie', movies_list)

def translate_text_except_titles(text):
    import re

    # Find all titles within curly braces
    titles = re.findall(r'\{(.*?)\}', text)

    # Replace titles with placeholders
    placeholder_text = re.sub(r'\{(.*?)\}', '{}', text)

    # Translate the non-title text
    translated_text = GoogleTranslator(source='auto', target='my').translate(placeholder_text)

    # Reinsert titles into the translated text
    for title in titles:
        translated_text = translated_text.replace('{}', title, 1)

    return translated_text

def translate_to_burmese(text):
    translator = GoogleTranslator(source='auto', target='my')
    return translator.translate(text)

# Generate prompt and get recommendation
prompt = f"Three Recommend similar movies with year title to '{select_value}' with very short description"
if st.button('Show Recommend'):
    # Generate response from Cohere
    response = co.generate(
        model='command-xlarge-nightly',  # Use a valid model ID
        prompt=prompt,
        max_tokens=200,
        temperature=0.9
    )

    response_text = response.generations[0].text

    # Translate the text while preserving movie titles
    translated_text = translate_text_except_titles(response_text)
    #translated_text = translate_to_burmese(response.generations[0].text)
    st.write(translated_text)
    # st.write(response.generations[0].text)

    # Get recommendations
    movie_names, movie_posters, movie_urls = recommend(select_value)
    columns = st.columns(len(movie_names))  # Dynamically create columns based on the number of movies

    for i, col in enumerate(columns):
        with col:
            col.markdown(
                f'''
                <div class="poster-container">
                    <a href="{movie_urls[i]}" target="_blank">
                        <img src="{movie_posters[i]}" style="width: 200px; height: 150px;
                        object-fit: cover;" />
                    </a>
                    <div class="poster-title">{movie_names[i]}</div>
                </div>
                ''',

                unsafe_allow_html=True
            )



#f'''
# <div style="text-align: center;" class="poster-container">
#     <a href="{movie_urls[i]}" target="_blank">
#         <img src="{movie_posters[i]}" style="width: 200px; height: 250px;
#         object-fit: cover; border-radius: 10px;" />
#     </a>
#     <div>{movie_names[i]}</div>
# </div>
# ''',
# unsafe_allow_html=True
# )

In [ ]:
!streamlit run ParadiseIO_Movie_Recommendation_System.py